# Установим openai через PIP

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


# Настраиваем openai
Подставить ключ сюда


In [2]:
import os
from openai import OpenAI
import pandas as pd

client = OpenAI(
    # This is the default and can be omitted
    api_key=""
    #api_key=os.getenv("OPENAI_API_KEY")
)



#Получим данные


In [3]:
from google.colab import files

# Загрузить файл с локального компьютера
uploaded = files.upload()

Saving data.csv to data.csv


In [4]:
data=pd.read_csv("data.csv")
data.head()
data.message

0      Привет! Я хочу заказать букет роз доставить в ...
1      Привет, хочу заказать доставку цветов в Новоси...
2      Здравствуйте, сколько стоит доставка цветов в ...
3      Здравствуйте, сколько стоит доставка цветов в ...
4      Привет! Хочу заказать цветы на день рождения п...
                             ...                        
115    Привет, доставите букет роз в г. екат? Срочно ...
116    Привет, сколько стоит доставка цветов в г. ниж...
117    Хочу цветы в Новосибирске. Скорее быстро сдела...
118    Здравствуйте! Хочу заказать букет роз с достав...
119    Здравствуйте. Я хочу заказать цветы в Москву н...
Name: message, Length: 120, dtype: object

# Делаем запросы
Для этого нужно использовать класс openai.ChatCompletion
В каждом сообщении надо передавать роль и само сообщение. Роль user означает что сообщение от пользователя, а роль assistant - от ChatGPT

In [5]:
def get_answer(data):
  gens=[]
  for i in data.message:
    prompt = f"{i}"
    prompt_template=f'''Перед тобой сообщение человека который хочет заказать цветы.О каком городе в сообщении идет речь? Напиши только официальное название города с большой буквы.Только название и ничего больше
    {prompt}'''

    messages = [{"role": "user", "content": prompt_template}]

    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt_template,
        }
    ],
    model="gpt-3.5-turbo",
)
    gens.append({prompt:response.choices[0].message.content})
  return gens

In [6]:
answer=get_answer(data)

In [7]:
answer

[{'Привет! Я хочу заказать букет роз доставить в Питер сегодня до 15:00. Можно у вас такое оформить?': 'Питер'},
 {'Привет, хочу заказать доставку цветов в Новосибирск, сколько это будет стоить?': 'Новосибирск'},
 {'Здравствуйте, сколько стоит доставка цветов в г. Екатеринбурге?': 'Екатеринбург'},
 {'Здравствуйте, сколько стоит доставка цветов в город Екб? Нужно до завтра. Спасибо.': 'ЕКАТЕРИНБУРГ'},
 {'Привет! Хочу заказать цветы на день рождения подруги в г. НСК, можете порекомендовать что-нибудь яркое и красивое? Спасибо!#': 'Новосибирск'},
 {'4. Добрый день! Я бы хотел заказать цветы для своей девушки в Нижнем. Можете сделать доставку с хорошим поздравлением?': 'Нижний.'},
 {'Добрый день, доставка цветов в Астану возможна сегодня? Нужен букет на юбилей.': 'Астана'},
 {'Привет, нужно срочно заказать цветы на день рождения друга в Нижнем Новгороде, вы сможете помочь?': 'Нижний Новгород'},
 {'Здравствуй! Сделайте мне красивый букет роз и орхидей и доставьте в Новосибирск, пожалуйста.'

openai справляется лучшее но все еще не идеально.Проблема с регистром решается легко ,можно все привести к нижнему регистру через lower. Но вот над тем что он иногда пишет питер или нижний надо работать больше


попробуем добавить систем промпт


In [11]:
def get_answer_system(data):
    gens = []
    for i in data.message:
        prompt = f"{i}"

        messages = [
            {"role": "system", "content": "Here is a message from a person who wants to order flowers.What city is the message referring to? Write only the official name of the city.Answer on russian"},
            {"role": "user", "content": prompt}
        ]

        response = client.chat.completions.create(
            messages=messages,
            model="gpt-3.5-turbo",
        )
        gens.append({prompt: response.choices[0].message.content})
    return gens

In [12]:
answer_system=get_answer_system(data)


In [ ]:
answer_system

пробежавшись глазами по полученным ответам я не нашел ни одного неправильного

In [36]:

prompts=[]
answers=[]

for i in answer_system:
  key=list(i.keys())[0]
  value=list(i.values())[0]
  prompts.append(key)
  answers.append(value)


ans_sys_df = pd.DataFrame({"prompt": prompts, "answers":answers} )



ans_sys_df.to_csv("answers.csv", index=False)

In [35]:
ans_sys_df.head()

,prompt,answers
0,Привет! Я хочу заказать букет роз доставить в ...,Санкт-Петербург
1,"Привет, хочу заказать доставку цветов в Новоси...",Новосибирск
2,"Здравствуйте, сколько стоит доставка цветов в ...",Екатеринбург
3,"Здравствуйте, сколько стоит доставка цветов в ...",Екатеринбург
4,Привет! Хочу заказать цветы на день рождения п...,Новосибирск
